In [1]:
import os

In [2]:
pwd

'c:\\Users\\slkpd\\Name-Entity-Recognition-Using-BERT\\research'

In [3]:
os.chdir("../")

In [4]:
pwd

'c:\\Users\\slkpd\\Name-Entity-Recognition-Using-BERT'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    #evaluation_strategy = IntervalStrategy.STEPS, # "steps"
    eval_steps: int, # Evaluation and Save happens every 50 steps
    save_total_limit: int, # Only last 5 models are saved. Older ones are deleted.
    learning_rate: float,
    per_device_train_batch_size: int,
    per_device_eval_batch_size: int,
    num_train_epochs: int,
    weight_decay: float,
    push_to_hub: bool,
    metric_for_best_model: str,
    load_best_model_at_end: bool

In [7]:
from src.nerBERT.constants import *
from src.nerBERT.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt = config.model_ckpt,
            eval_steps = params.eval_steps,
            num_train_epochs = params.num_train_epochs,
            learning_rate = params.learning_rate
            per_device_train_batch_size = params.per_device_train_batch_size,
            per_device_eval_batch_size = params.per_device_eval_batch_size,
            weight_decay = params.weight_decay,
            push_to_hub = params.push_to_hub,
            metric_for_best_model = params.metric_for_best_model,
            load_best_model_at_end = params.load_best_model_at_end
        )

        return model_trainer_config

In [10]:
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizerFast
from transformers import DataCollatorForTokenClassification
from transformers import AutoModelForTokenClassification
from datasets import load_dataset, load_from_disk
import torch
from transformers import EarlyStoppingCallback, IntervalStrategy

c:\Users\slkpd\anaconda3\envs\NER_BERT\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config


    
    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = BertTokenizerFast.from_pretrained(self.config.model_ckpt)
        model_bert = AutoModelForTokenClassification.from_pretrained(self.config.model_ckpt).to(device)
        classification_data_collator = DataCollatorForTokenClassification(tokenizer, model=model_pegasus)
        
        #loading data 
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        # trainer_args = TrainingArguments(
        #     output_dir=self.config.root_dir, num_train_epochs=self.config.num_train_epochs, warmup_steps=self.config.warmup_steps,
        #     per_device_train_batch_size=self.config.per_device_train_batch_size, per_device_eval_batch_size=self.config.per_device_train_batch_size,
        #     weight_decay=self.config.weight_decay, logging_steps=self.config.logging_steps,
        #     evaluation_strategy=self.config.evaluation_strategy, eval_steps=self.config.eval_steps, save_steps=1e6,
        #     gradient_accumulation_steps=self.config.gradient_accumulation_steps
        # ) 


        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir, 
            evaluation_strategy = IntervalStrategy.STEPS, # "steps"
            eval_steps = 50, # Evaluation and Save happens every 50 steps
            save_total_limit = 5, # Only last 5 models are saved. Older ones are deleted.
            learning_rate=2e-5,
            per_device_train_batch_size=16,
            per_device_eval_batch_size=16,
            num_train_epochs=5,
            weight_decay=0.01,
            push_to_hub=False,
            metric_for_best_model = 'f1',
            load_best_model_at_end=True
        ) 

        trainer = Trainer(model=model_bert, args=trainer_args,
                  tokenizer=tokenizer, data_collator=classification_data_collator,
                  callbacks = [EarlyStoppingCallback(early_stopping_patience=3)],
                  train_dataset=dataset_samsum_pt["train"], 
                  eval_dataset=dataset_samsum_pt["validation"])
        
        trainer.train()

        ## Save model
        model_bert.save_pretrained(os.path.join(self.config.root_dir,"bert-base-uncased-model"))
        ## Save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))

SyntaxError: positional argument follows keyword argument (3322846828.py, line 26)